# Blöcke und Blockchain

## Allgemein

Alle durchgeführten Transaktionen werden in einer Art Datenbank - der Blockchain - abgespeichert. Der wesentliche Unterschied zwischen der Blockchain und einer herkömmlichen Datenbank besteht in der Dezentralität. Um ein möglichst effizientes Fortschreiben dieser dezentralen Struktur zu ermöglichen, werden Updates der Blockchain in sogennanten Blöcken durchgeführt.  
Ein Block besteht aus Transaktionen, die vom Ersteller des Blocks - dem Miner - ausgewählt/gebündelt werden. Jeder Block beinhaltet zusätzlich zu den Transaktionen ein Feld mit dem Hash des vorangegangenen Blocks in der Blockchain. Dadurch entsteht die Kettenstruktur, d.h. innerhalb der Blockchain hat jeder Block seine feste Position. Um sicherzustellen, dass nachträgliche Veränderungen in der Historie der Blockchain (d.h. insbesondere eine nachträgliche Manipulation der Besitzverhältnisse) nur durch den Einsatz von sehr viel Rechenleistung durchgeführt werden könnten, muss bei der Erzeugung eines Blocks die Lösung eines numerischen Rätsels im Block hinterlegt werden.


## Block


#### Implementierung
1. Jeder Block enthält eine Referenz auf den Hashwert des vorangegangenen Blocks (`previous_hash`), einen Zeitstempel (`timestamp`) und den Hashwert von sich selbst (`id`). 
2. Jeder Block enthält ein sogenanntes `nonce` Feld. Der Hintegrund des Felds ist Folgender: Das numerische Rätsel, das vom Miner gelöst werden muss, lässt sich wie folgt beschreiben: "Finde einen Blockhash, der mit einer vorgegeben Anzahl an Nullen beginnt." Durch die Unumkehrbarkeit der Hashfunktion ist die effizieteste Methode dieses Rätsel zu lösen ganz einfach Ausprobieren. Ausprobieren bedeutet, dass der Block durch arbiträre Veränderung des Inhalts im `nonce` Feld verändert wird und im Anschluss der Hash des Blocks berechnet wird. Dies wird iterativ solange durchgeführt, bis ein Hash mit vorgegebener Anzahl an Nullen gefunden wurde.  
3. Damit effizient auf die UTXOs zugegriffen werden kann, erzeugen wir bei der Konstrukttion eines Blocks zusätzlich noch ein `output_transaction_mapping`. Dies ist einfach ein OrderedDict mit `TransactionOutput.id:Transaction.id`. Mit Hilfe des `output_transaction_mapping` können wir später einen bestimmten UTXO innerhalb eines Blocks schnell finden. Das Mapping wird mit der Methode `create_output_transaction_mapping` erzeugt die im Konstruktor aufgerufen wird. Der Zugriff auf die UTXOS erfolgt über die `get_output_by_id` Methode. `get_output_by_id` gibt für eine übergebene `output_id` das entsprechende `TransactionOutput` Objekt aus dem Block zurück (sofern dieses vorhanden ist).
4. Jeder Block enthält zusätzlich eine `verify_block(self, blockchain)` Funktion, die die Validität des Blocks gegeben einer bestimmten 'blockchain' überprüft. Die Methode wird zunächst als Dummy implementiert, so dass zunächst immer der Wert `True` zurückgegeben wird. Die tatsächliche Implementierung von `verif_block` folgt im nächsten Notebook.

*Zusatzinfo: Wir fügen die Verifizierung bzgl. der Gültigkeit eines Blocks (und auch der Gültigkeit einer Transaktion) später hinzu. Zuerst implementieren wir lediglich das Grüst der Klasse um dieses Gerüst zu einem späteren Zeitpunkt dann zu erweitern.*

In [5]:
# Automatically relaoad modules that have changed while
# being loaded.
%load_ext autoreload
# Set equal to 2 to enable auto realod,
# set equal to 0 to disable auto relaod.
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
# Import stuff

# Import the modules we need
from IPython.core.debugger import set_trace

from time import time

from collections import OrderedDict
import json
from pprint import pprint

# This is the outsourced code from the 
# previous section.
from FEDCoin import *
from utils import *


In [18]:
class Block:
    def __init__(self, transactions, previous_hash, timestamp=None, nonce=None):

        # If timestamp is empty, this indicates the creation of a 'real'
        # new block. 
        if timestamp is None:
            self.timestamp = time()
        else:
            self.timestamp = timestamp

        self.previous_hash = previous_hash

        # OrderedDict
        # Transaction.id:Transaction
        self.transactions = transactions

        # If nonce is empty, this indicates the creation of a 'real'
        # new block. (Goes hand in hand with timestamp)     
        if nonce is None:
            self.nonce = 0
        else:
            self.nonce = nonce

        # Mapping to get efficient access to an TransactionOutput
        # The mapping doesn't contain any additional info, 
        # so it is not part of the actual block information. 
        # This means it is not used to create the hash later on.
        self.output_transaction_mapping = self.create_output_transaction_mapping()

        self.id = hash_stuff(self.odict_block())

    def odict_block(self):
        # Creates an OrderedDict of the Block info that is
        # used to calculate the hash.

        # Create an OrderedDict of all Transactions
        # by looping through the set of Transactions
        # and converting them each to an OrderedDict.
        # Then put all the OrderedDict of the Transactions
        # into one large OrderedDict.
        transaction_odict = OrderedDict()
        for transaction in self.transactions.values():
            transaction_odict[transaction.id] = transaction.get_full_odict()

        block_odict = OrderedDict({'timestamp': self.timestamp,
                                   'transactions': transaction_odict,
                                   'previous_hash': self.previous_hash,
                                   # Note that the next line --as mentioned--
                                   # does not need to be part of the OD.
                                   # 'output_transaction_mapping': self.output_transaction_mapping,
                                   'nonce': self.nonce})
        return block_odict


    def get_full_odict(self):
        # Return the full OrderedDict of the
        # block (including the Hash)

        response = self.odict_block()
        response['id'] = self.id
        return response

    def create_output_transaction_mapping(self):
        # TODO:
        # Creates the output_transaction_mapping.
        # The method returns an OrderedDict of with
        # TransactionOutput.id:Transaction.id.
        # Therefore, calling output_transaction_mapping[some_ta_id]
        # returns the id of the Transaction which contains this
        # specific output.
        output_transaction_mapping = OrderedDict()
        for transaction_id, transaction in self.transactions.items():
            for output_id in transaction.outputs.keys():
                output_transaction_mapping[output_id] = transaction_id
        return output_transaction_mapping

    def get_output_by_id(self, output_id):
        # Utilizes the output_transaction_mapping.
        # Returns the TransactionOutput for a given
        # TransactionOutput.id.

        transaction = self.transactions[self.output_transaction_mapping[output_id]]
        return transaction.outputs[output_id]

    def verify_block(self, blockchain):
        return True


#### TODO: Test der Block Klasse

Im nächsten Schritt soll jeder selbst versuchen die Implementierung der Klasse zu testen. Es geht einfach nur darum zu übeprüfen, ob die entsprechenden Methoden funktionieren und das tun was sie sollen. Geht hierfür am besten folgendermaßen vor:

1. Erzeuge zwei `Wallets` *w_alice* und *w_bob*, damit Du gültige Keys für die Erzeugung einer `Transaction` hast.
2. Erzeuge drei `TransactionOutputs` *to_a*,*to_b* und *to_c*. Für den `value` sollte gelten: *to_a* = *to_b* + *to_c*. *to_a* und *to_c* sollen *w_alice* gehören, *to_b* gehört *w_bob*. *Hinweis: Wir modellieren hier den klassichen Fall *w_alice* bezahlt ein Gut bei *w_bob *und überweist das Wechselgeld wieder an sich selbst zurück.*
3. Erzeuge die dazu passende `Transaction` *t_alice_to_bob* und signiere sie mit dem `private_key` von *w_alice*.
4. Wiederhole Schritt 2. und 3. um eine zweite `Transaction` von *w_bob* zu *w_alice* zu erzeugen. Verwende hierfür komplett neue `TransactionOutput`s *to_d*, *to_e*, *to_f*.
5. Erzeuge einen `Block` mit den beiden `Transaction`s. Das Feld `nonce` und den `timestamp` bei der Erzeugung einfach leer lassen (das heißt nichts übergeben --> Defaultwerte werden verrwendet). Für das Feld `previous_hash` einfach 'abc' angeben (zum Test).
6. Teste, ob die Methoden der `Block` Klasse funktionieren bzw., ob der `Block` korekt erzeugt wurde. Hierfür macht es Sinn sich das Objekt als `OrderedDict` ausgeben zu lassen (mit der `get_full_odict()` Methode). Das `output_transaction_mapping` muss selbstverständlich zusätzlich überprüft werden, weil es nicht Teil des `OrderedDict` des Blocks ist.

In [27]:
# Give it a try!:

# Step 1
w_alice = Wallet()
w_bob = Wallet()

# Step 2
to_a = TransactionOutput(w_alice.public_key, 20)
to_b = TransactionOutput(w_bob.public_key, 15)
to_c = TransactionOutput(w_alice.public_key, 5)

# Step 3
outputs_od = OrderedDict({
    to_b.id: to_b,
    to_c.id: to_c
})

t_alice_to_bob = Transaction(w_alice.public_key, [to_a.id], outputs_od)

# Step 4
to_d = TransactionOutput(w_bob.public_key, 20)
to_e = TransactionOutput(w_alice.public_key, 15)
to_f = TransactionOutput(w_bob.public_key, 5)

outputs_od = OrderedDict({
    to_e.id: to_e,
    to_f.id: to_f
})

t_bob_to_alice = Transaction(w_bob.public_key, [to_d.id], outputs_od)

# Step 5
transactions = OrderedDict({t_alice_to_bob.id: t_alice_to_bob, t_bob_to_alice.id: t_bob_to_alice})
new_block = Block(transactions, "abc")

# Step 6
if new_block.get_output_by_id(to_c.id) == to_c:
    print('Mapping seems to work.')
print('\n')
print(new_block.output_transaction_mapping)
print('\n')
# pprint(new_block.get_full_odict())

Mapping seems to work.


OrderedDict([('dbea2d567418f8fc731189539f98645954da1378c094685b6af173430f102b64', '213bbf99982c37033fb684484506712c431c82c4ec7bdf986b98c857a80e3a87'), ('efd3078bc83c03e183e3ff4eba1cc208d26e6c1e93ddc7a51ddca55e892cd930', '213bbf99982c37033fb684484506712c431c82c4ec7bdf986b98c857a80e3a87'), ('c4e3ed3384bb99b5dda8b9274abe66221a59845c54cea22ab1a35e653422ecdb', 'c1df41ea5f97d1d530acb91e95c4d40c7ee7bbea1befc643ac060ca9cf7c8570'), ('a7bf32db8a14bffec0923e44982ae792b287c17ba3f2acc6acf9d05f63d493bb', 'c1df41ea5f97d1d530acb91e95c4d40c7ee7bbea1befc643ac060ca9cf7c8570')])




## Blockchain


### Allgemein
Die eigentliche Blockchain besteht nun einfach aus der Aneinanderreihung der eben eingeführten Blöcken. Da es für die Lösung des Mining-Rätsels bei der Blockerstellung an Rechenleistung seitens des Miners bedarf, wird der Miner für seinen Einsatz bei erfolgreicher Erstellung eines Blocks durch einen *Mining Reward* belohnt. Dies geschieht durch das *Schürfen* neuer *FEDCoins* (kurz FC). Die damit verbundene Transaktion wird als *Coinbase Transaktion* bezeichnet. Im Rahmen unserer Kryptowährung setzten wir den initialen *Mining Reward* auf 5 FC. Um zu verhindern, dass es irgendwann eine unedlich große Anzahl an FC gibt, legen wir außerdem fest, dass dieser Reward alle 10 Blöcke halbiert werden soll. Die maximale Blockgröße wird auf 5 Transaktionen festegesezt (5 bedeutet bei uns 5 + 1 *Coinbase Transaktion*). Außerdem legen wir die Schwierigkeit des numerischen Rätsels fest indem gefordert wird, dass ein gültiger Blockhash mit mindestens 5 Nullen beginnen muss. 
<br/>Zusätzlich zu der eigentlichen Blockchain (damit ist die Aneinanderreihung der Blöcke gemeint, im Folgenden auch kurz *Chain* genannt) verwaltet ein `Blockchain` Objekt auch einen *Mempool* (für Memory Pool). Im *Mempool* werden alle der Blockchain bekannten Transaktionen, die vor dem Hintegrund der in der *Chain* hinterlegten Besitzverhältnisse gültig wären aber bisher noch nicht Teil eines Blocks in der Blockchain sind, hinterlegt. Da bzgl. der Transaktionen im *Mempool* noch kein *Proof-of-Work* investiert wurde, sind diese Transaktion in gewisser weiße noch manipulierbar (dazu später mehr). Bei der Erstellung eines neuen Blocks bedient sich der Miner immer aus Transaktionen, die im Mempool der entsprechenden Blockchain hinterlegt sind.
<br/>Der dritte wesentliche Bestandteil eines `Blockchain` Objektes, neben *Chain* und *Mempool*, ist eine Liste der aktuell noch nicht ausgegebenen `TransactionOutputs` kurz auch *UTXOs* (für *Unspent Transaction Outputs*). Theoretisch könnte man diese Liste auch zur Laufzeit erstellen, das wäre aber mit extrem viel Rechenaufwand verbunden und es ist deshalb effizienter diese Liste zum Teil der Datenstruktur zu machen.

### Implementierung
Der *Genesisblock* (der erste Block jedes Blockchain Objekts) wird immer direkt bei Initialisierung eines `Blockchain` Objekts hinzugefügt. Er ist deshalb ein spezieller Block, der *hardcoded* in der Implementierung unserer `Blockchain` hinterlegt wird. D.h. egal wer eine neue Blockchain erstellt, der Genesisblock muss immer genau so wie hinterlegt aussehen, ansonten ist die Blockchain nicht valide (mehr zu Validität später).

1. **Klassenvariablen:** Die das *Mining* betreffenden Paramter sollen als Klassenvariablen der `Blockchain` Klasse hinterlegt werden. D.h. wir haben insgesamt 4 Klassenvariablen:
    * `initial_difficulty`
    * `initial_block_reward`
    * `half_time_in_blocks`
    * `max_block_size_in_transactions` 
    <br/>
    <br/>
2. **Konstruktur:** Jedes `Blockchain` Objekt soll die folgenden vier OrderedDicts  haben (bei Initialisierung sollen diese alle leer sein - Info in Klammern deshalb lediglich zum besseren Verständnis):
    * `chain` (Block.id:Block)
    * `UTXOs` (TransactionOutput.id:Block.id)
    * `mempool` (Transaction.id:Transaction)
    * `mempool_UTXOs` (TransactionOutput.id:Transaction.id)
    <br/>
    <br/>
   Im `mempool_UTXOs` werden die *UTXOs* gespeichert, die in den Transaktionen aus dem *Mempool* verwendet werden. Das hat den Vorteil, dass für eine `Blockchain` bei der Aufnahme einer neuen Transaktion in den *Mempool* einfach überprüft werden kann, ob bereits eine konkurierende Transaktion, die den gleichen *UTXO* verwendet, im Mempool ist. Sollte dies der Fall sein, macht es Sinn die neue Transaktion nicht in den *Mempool* aufzunehmen.  
   Bei der Initialisierung des `Blockchain` Objekts soll außerdem die Methode `add_genesis_block` aufgerufen werden.  
    Außerdem soll jedes `Blockchain` Objekt den aktuellen *Proof-of-Work* (PoW), der in der gesamten Chain steckt, als Eigenschaft haben. D.h. im Konstruktur muss eine Variable `self.pow = self.initial_difficulty * len(self.chain.keys())` initialisiert werden. Das erleichtert später die Identifikation der Kette mit dem meisten PoW.
<br/>
    <br/>    
3. **`create_gen_block(self)`:** Der *Genesisblock* (erster Block jedes Blockchain-Objekts) wird hardcoded hinterlegt. Folgende Schritte sind hierfür zu implementieren:
    1. Der *public_key* der Genesis Wallet (`genesis_pub_key`) lautet: `3059301306072a8648ce3d02...`
    2. Ein `TransactionOutput` mit dem Ziel `genesis_pub_key`, `value = initial_block_reward`, `random = 0` und `timestamp = 0` soll erzeugt werden.
    3. Eine Transaktion mit diesem `TransactionOutput` soll erzeugt werden, für die Transaktion soll `sender = 'coinbase'`, `inputs = ['coinbase_UTXO']` und `timestamp = 0` sein
    4. Ein `Block` mit ausschließlich dieser Transaktion soll erzeugt werden. Für diesen `Block` soll gelten: `timestamp = 0`, `previous_hash = 'genesis'` und `nonce = 51473786`. Dieses `Block` Objekt soll von der Methode zurückgegeben werden.
<br/>
    <br/>
4. **`add_block(self, new_block)`:** Die Methode soll den `new_block` zur Blockchain hinzufügen. Nach dem Hinzufügen eines neuen Blocks wird `self.UTXOs` durch eine weitere Methode `update_UTXOs(new_block)` aktualisiert und zusätzlich wird  `self.mempool` durch eine weitere Methode `update_mempool(new_block)` ebenfalls aktualisiert. Außerdem muss der Proof-of-Work `self.pow` der Chain beim Hinzufügen eines Blocks aktualisiert werden. Für den Fall, dass der neue Block der *Genesisblock* ist, kann auf das *Mempool* Update verzichtet werden. Wird `new_block` durch die Methode erfolgreich hinzugefügt, ist der Returnwert `True`, andernfalls `False`.
<br/>
    <br/>
5. **`update_UTXOs(self, new_block)`:** Fügt neue *UTXOs* zu `self.UTXOs` hinzu und wirft verbrauchte *UTXOs* aus `self.UTXOs` raus.
<br/>
    <br/>
6. **`update_mempool(self, new_block)`:** Entfernt die im `new_block` enthaltenen Transaktionen aus `self.mempool`.
<br/>
    <br/>
7. **`add_genesis_block(self)`:** Führt `create_gen_block` und `add_block` mit dem *Genesisblock* aus.
<br/>
    <br/>
8. **`get_UTXO_by_id(self,UTXO_id)`:** Gibt den UTXO zu einer gegebenen UTXO_id zurück.
<br/>
    <br/>
9. **`get_last_block(self)`:** Gibt den letzten Block (der Aktuellste) der Chain zurück.

## Blockchain


### Allgemein
Die eigentliche Blockchain besteht nun einfach aus der Aneinanderreihung der eben eingeführten Blöcken. Da es für die Lösung des Mining-Rätsels bei der Blockerstellung an Rechenleistung seitens des Miners bedarf, wird der Miner für seinen Einsatz bei erfolgreicher Erstellung eines Blocks durch einen *Mining Reward* belohnt. Dies geschieht durch das *Schürfen* neuer *FEDCoins* (kurz FC). Die damit verbundene Transaktion wird als *Coinbase Transaktion* bezeichnet. Im Rahmen unserer Kryptowährung setzten wir den initialen *Mining Reward* auf 5 FC. Um zu verhindern, dass es irgendwann eine unedlich große Anzahl an FC gibt, legen wir außerdem fest, dass dieser Reward alle 10 Blöcke halbiert werden soll. Die maximale Blockgröße wird auf 5 Transaktionen festegesezt (5 bedeutet bei uns 5 + 1 *Coinbase Transaktion*). Außerdem legen wir die Schwierigkeit des numerischen Rätsels fest indem gefordert wird, dass ein gültiger Blockhash mit mindestens 5 Nullen beginnen muss. 
<br/>Zusätzlich zu der eigentlichen Blockchain (damit ist die Aneinanderreihung der Blöcke gemeint, im Folgenden auch kurz *Chain* genannt) verwaltet ein `Blockchain` Objekt auch einen *Mempool* (für Memory Pool). Im *Mempool* werden alle der Blockchain bekannten Transaktionen, die vor dem Hintegrund der in der *Chain* hinterlegten Besitzverhältnisse gültig wären aber bisher noch nicht Teil eines Blocks in der Blockchain sind, hinterlegt. Da bzgl. der Transaktionen im *Mempool* noch kein *Proof-of-Work* investiert wurde, sind diese Transaktion in gewisser weiße noch manipulierbar (dazu später mehr). Bei der Erstellung eines neuen Blocks bedient sich der Miner immer aus Transaktionen, die im Mempool der entsprechenden Blockchain hinterlegt sind.
<br/>Der dritte wesentliche Bestandteil eines `Blockchain` Objektes, neben *Chain* und *Mempool*, ist eine Liste der aktuell noch nicht ausgegebenen `TransactionOutputs` kurz auch *UTXOs* (für *Unspent Transaction Outputs*). Theoretisch könnte man diese Liste auch zur Laufzeit erstellen, das wäre aber mit extrem viel Rechenaufwand verbunden und es ist deshalb effizienter diese Liste zum Teil der Datenstruktur zu machen.

### Implementierung
Der *Genesisblock* (der erste Block jedes Blockchain Objekts) wird immer direkt bei Initialisierung eines `Blockchain` Objekts hinzugefügt. Er ist deshalb ein spezieller Block, der *hardcoded* in der Implementierung unserer `Blockchain` hinterlegt wird. D.h. egal wer eine neue Blockchain erstellt, der Genesisblock muss immer genau so wie hinterlegt aussehen, ansonten ist die Blockchain nicht valide (mehr zu Validität später).

1. **Klassenvariablen:** Die das *Mining* betreffenden Paramter sollen als Klassenvariablen der `Blockchain` Klasse hinterlegt werden. D.h. wir haben insgesamt 4 Klassenvariablen:
    * `initial_difficulty`
    * `initial_block_reward`
    * `half_time_in_blocks`
    * `max_block_size_in_transactions` 
    <br/>
    <br/>
2. **Konstruktur:** Jedes `Blockchain` Objekt soll die folgenden vier OrderedDicts  haben (bei Initialisierung sollen diese alle leer sein - Info in Klammern deshalb lediglich zum besseren Verständnis):
    * `chain` (Block.id:Block)
    * `UTXOs` (TransactionOutput.id:Block.id)
    * `mempool` (Transaction.id:Transaction)
    * `mempool_UTXOs` (TransactionOutput.id:Transaction.id)
    <br/>
    <br/>
   Im `mempool_UTXOs` werden die *UTXOs* gespeichert, die in den Transaktionen aus dem *Mempool* verwendet werden. Das hat den Vorteil, dass für eine `Blockchain` bei der Aufnahme einer neuen Transaktion in den *Mempool* einfach überprüft werden kann, ob bereits eine konkurierende Transaktion, die den gleichen *UTXO* verwendet, im Mempool ist. Sollte dies der Fall sein, macht es Sinn die neue Transaktion nicht in den *Mempool* aufzunehmen.  
   Bei der Initialisierung des `Blockchain` Objekts soll außerdem die Methode `add_genesis_block` aufgerufen werden.  
    Außerdem soll jedes `Blockchain` Objekt den aktuellen *Proof-of-Work* (PoW), der in der gesamten Chain steckt, als Eigenschaft haben. D.h. im Konstruktur muss eine Variable `self.pow = self.initial_difficulty * len(self.chain.keys())` initialisiert werden. Das erleichtert später die Identifikation der Kette mit dem meisten PoW.
<br/>
    <br/>    
3. **`create_gen_block(self)`:** Der *Genesisblock* (erster Block jedes Blockchain-Objekts) wird hardcoded hinterlegt. Folgende Schritte sind hierfür zu implementieren:
    1. Der *public_key* der Genesis Wallet (`genesis_pub_key`) lautet: `3059301306072a8648ce3d02...`
    2. Ein `TransactionOutput` mit dem Ziel `genesis_pub_key`, `value = initial_block_reward`, `random = 0` und `timestamp = 0` soll erzeugt werden.
    3. Eine Transaktion mit diesem `TransactionOutput` soll erzeugt werden, für die Transaktion soll `sender = 'coinbase'`, `inputs = ['coinbase_UTXO']` und `timestamp = 0` sein
    4. Ein `Block` mit ausschließlich dieser Transaktion soll erzeugt werden. Für diesen `Block` soll gelten: `timestamp = 0`, `previous_hash = 'genesis'` und `nonce = 51473786`. Dieses `Block` Objekt soll von der Methode zurückgegeben werden.
<br/>
    <br/>
4. **`add_block(self, new_block)`:** Die Methode soll den `new_block` zur Blockchain hinzufügen. Nach dem Hinzufügen eines neuen Blocks wird `self.UTXOs` durch eine weitere Methode `update_UTXOs(new_block)` aktualisiert und zusätzlich wird  `self.mempool` durch eine weitere Methode `update_mempool(new_block)` ebenfalls aktualisiert. Außerdem muss der Proof-of-Work `self.pow` der Chain beim Hinzufügen eines Blocks aktualisiert werden. Für den Fall, dass der neue Block der *Genesisblock* ist, kann auf das *Mempool* Update verzichtet werden. Wird `new_block` durch die Methode erfolgreich hinzugefügt, ist der Returnwert `True`, andernfalls `False`.
<br/>
    <br/>
5. **`update_UTXOs(self, new_block)`:** Fügt neue *UTXOs* zu `self.UTXOs` hinzu und wirft verbrauchte *UTXOs* aus `self.UTXOs` raus.
<br/>
    <br/>
6. **`update_mempool(self, new_block)`:** Entfernt die im `new_block` enthaltenen Transaktionen aus `self.mempool`.
<br/>
    <br/>
7. **`add_genesis_block(self)`:** Führt `create_gen_block` und `add_block` mit dem *Genesisblock* aus.
<br/>
    <br/>
8. **`get_UTXO_by_id(self,UTXO_id)`:** Gibt den UTXO zu einer gegebenen UTXO_id zurück.
<br/>
    <br/>
9. **`get_last_block(self)`:** Gibt den letzten Block (der Aktuellste) der Chain zurück.

In [40]:
class Blockchain:
    initial_difficulty = 3
    initial_block_reward = 50
    half_time_in_blocks = 10
    max_block_size_in_transactions = 5

    def __init__(self):
        self.chain = OrderedDict()  # OD containing the blocks of the chain - Block.id:Block
        self.UTXOs = OrderedDict()  # OD with UTXOs - TransactionOutput.id:Block.id
        self.mempool = OrderedDict()  # OD with transactions - Transaction.id:Transaction
        self.mempool_UTXOs = OrderedDict()  # OD with UTXOs used in mempool - TransactionOutput.id:Transaction.id
        self.pow = 0  # pow of the chain
        self.add_genesis_block()

    def create_genesis_block(self):
        # Create Genesis Transaction
        genesis_pub_key = '3059301306072a8648ce3d020106082a8648ce3d030107034200043f3bd6d16ce4bde95a8237170aaa106388485498234a3dca5d0c273d907c03c3e72017bec13cf6e893e96da0f9d6c7037c79b40cb006aff12ae88adae1b0bb7e'

        genesis_output = TransactionOutput(genesis_pub_key, self.initial_block_reward, 0, timestamp=0)
        genesis_transaction = Transaction('coinbase', ['coinbase_UTXO'],
                                          OrderedDict({genesis_output.id: genesis_output}), 0)

        # Create Genesis Block as block object
        genesis_block = Block(OrderedDict({genesis_transaction.id: genesis_transaction}), 'genesis', 0, 51473786)

        # Add the Genesis Block
        return genesis_block

    def add_block(self, new_block):
        # The first block has to be the genesis
        if not self.chain:
            if new_block.__eq__(self.create_genesis_block()):
                self.chain[new_block.id] = new_block
                # Update pow of the chain
                self.pow = self.initial_difficulty * len(self.chain.keys())
                # Update UTXOs
                self.update_UTXOs(new_block)
                return True
            else:
                return False
        else:
            # Check if block has correct previous_hash and if new block is valid
            if self.get_last_block().id == new_block.previous_hash and \
                    new_block.verify_block(self):
                self.chain[new_block.id] = new_block
                # Update pow of the chain
                self.pow = self.initial_difficulty * len(self.chain)
                # Update UTXOs
                self.update_UTXOs(new_block)
                # Update mempool
                self.update_mempool(new_block)

                return True
            else:
                return False

    def update_UTXOs(self, new_block):
        # TODO:
        # After a block has been added to the chain
        # the UTXOs are updated

        # remove all referenced inputs

        pass

    def update_mempool(self, new_block):
        # TODO:
        # After a block has been added to the chain remove 
        # its Transactions from the chain's mempool.

        # Remove the Transactions from that block in the mempool
        # Note there is no return value.

        pass

    def add_genesis_block(self):
        # Create and add the genesis to the chain

        genesis_block = self.create_genesis_block()
        self.add_block(genesis_block)

    def get_UTXO_by_id(self, UTXO_id):
        # Returns the UTXO for a given UTXO_id

        return (self.chain[self.UTXOs[UTXO_id]].get_output_by_id(UTXO_id))

    def get_last_block(self):
        # Returns the most recent added block of the chain

        last_id = next(reversed(self.chain))
        return (self.chain[last_id])


In [8]:
# Testing it:
bc = Blockchain()
# This should only contain the UTXOs_id with Block reference from the genesis Transaction (i.e. OD with one entry only):
pprint(bc.UTXOs)
print("\n")

# Get the Genesis UTXO by its id (should return the Genesis UTXO)
pprint(bc.get_UTXO_by_id('13f7b983fb68e4fde6b1b77df955098a2526fe62c965ad435fe61538832bcdb8').get_full_odict())
print("\n")

# Print the last block --> genesis in this case as OD
pprint(bc.get_last_block().get_full_odict())
print("\n")

# Print the pow of the chain (one block, so should be equal 3)
print(bc.pow)


OrderedDict([('13f7b983fb68e4fde6b1b77df955098a2526fe62c965ad435fe61538832bcdb8', '0000002dc75b3bc7ad4b5bce7d572f4d13982e8eb7a67dca6f50d9abeb7f6e81')])


OrderedDict([('time', 0), ('random', 0), ('recipient', '3059301306072a8648ce3d020106082a8648ce3d030107034200043f3bd6d16ce4bde95a8237170aaa106388485498234a3dca5d0c273d907c03c3e72017bec13cf6e893e96da0f9d6c7037c79b40cb006aff12ae88adae1b0bb7e'), ('value', 50), ('id', '13f7b983fb68e4fde6b1b77df955098a2526fe62c965ad435fe61538832bcdb8')])


OrderedDict([('timestamp', 0), ('transactions', OrderedDict([('c4e819034285642fd528308f4b71c2d4f059e58ccf4fea18cc99ec2eb19eb3cb', OrderedDict([('timestamp', 0), ('sender', 'coinbase'), ('inputs', ['coinbase_UTXO']), ('outputs', OrderedDict([('13f7b983fb68e4fde6b1b77df955098a2526fe62c965ad435fe61538832bcdb8', OrderedDict([('time', 0), ('random', 0), ('recipient', '3059301306072a8648ce3d020106082a8648ce3d030107034200043f3bd6d16ce4bde95a8237170aaa106388485498234a3dca5d0c273d907c03c3e72017bec13cf6e893e96da0f9